In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [30]:
#get Basic Probability Assignment
def getMass(sample, accuracy):
    bp = [[0,0,0,0],[0,0,0,0]]
    for i in range(len(sample)):
        if sample[i] == 'S':
            bp[0][i]=accuracy[i]
        else:
            bp[0][i]=1-accuracy[i]
        if sample[i] == 'M':
            bp[1][i]=accuracy[i]
        else:
            bp[1][i]=1-accuracy[i]   
    return bp

In [3]:
#get Normalized constant
def getK(bp):
    k=bp[0][0]*bp[0][1]+bp[1][0]*bp[1][1]
    return k

In [4]:
#Data fusion according to Dempster combination rules
def getDempster(k, bp):
    m = bp[0]*bp[1]/k
    return m

In [5]:
def doDempsterShafer(sample,accuracy,anum):
    #get Basic Probability
    BasicProbability = getMass(sample,accuracy)
    #print(BasicProbability)
    mS = 0
    mM = 0
    k =0
    bp_temp = [[0,0],[0,0]]
    #Ds Fusion
    for m in range(anum-1):
        if m==0 :
            bp_temp[0][0] = BasicProbability[0][0]
            bp_temp[1][0] = BasicProbability[1][0]
        else:
            bp_temp[0][0] = mS
            bp_temp[1][0] = mM
        bp_temp[0][1] = BasicProbability[0][m+1]
        bp_temp[1][1] = BasicProbability[1][m+1]
        
        k = getK(bp_temp)
        mS = getDempster(k,bp_temp[0])
        mM = getDempster(k,bp_temp[1])
    if mS>=0.5:
        return 'S'
    else:
        return 'M'

In [1]:
def DempsterShafer_116(accuracylist):
    data = pd.read_excel('middle\\phase_prediction_116.xlsx')

    data17 = data[['formula','LR_17','NB_17','RF_17','SVM_sigmoid_17']]
    dataSFS = data[['formula','LR_SFS','NB_SFS','RF_SFS','SVM_sigmoid_SFS']]
    Accuracy17 = accuracylist
    #LR NB RF sigmoid      
    #Getting the accuracy of each algorithm
    Accuracy17 = accuracylist[::2]
    AccuracySFS =accuracylist[1::2]
    
    AlgNum = len(Accuracy17)
   
    data_ds_17 = []
    data_ds_SFS = []
    formula = list(data17.iloc[:, 1])
    new_data_17 = data[['LR_17','NB_17','RF_17','SVM_sigmoid_17']].values
    new_data_SFS = data[['LR_SFS','NB_SFS','RF_SFS','SVM_sigmoid_SFS']].values
    for n in range(116):
        sample_17 = new_data_17[n]
        sample_SFS = new_data_SFS[n]
        data_ds_17.append(doDempsterShafer(sample_17,Accuracy17,AlgNum))
        data_ds_SFS.append(doDempsterShafer(sample_SFS,AccuracySFS,AlgNum))
        #data_ds[n][1] = doDempsterShafer(onesample,AccuracySFS,AlgNum)
    df17 = pd.DataFrame(data_ds_17, columns=['result_ds_17'])
    newdata = pd.concat([data17,df17],axis=1) 
    indexList = range(1, 117)
    newdata.insert(0, 'indexs', indexList, allow_duplicates=False)
    newdata.to_excel(excel_writer="ds_result_17_116.xlsx",index=False)
    
    dfsfs = pd.DataFrame(data_ds_SFS, columns=['result_ds_SFS'])
    SFS_data = pd.concat([dataSFS,dfsfs],axis=1) 
    SFS_data.insert(0, 'indexs', indexList, allow_duplicates=False)
    SFS_data.to_excel(excel_writer="ds_result_SFS_116.xlsx",index=False)